This notebook shows the reconstruction of a 3D GRE with two almost identical echotimes sampled with 3D Lissajous variable density shell trajectories.


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import sys

sys.path.insert(0, "../src")

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import torch
import zarr

from juart.conopt.functional.fourier import nonuniform_fourier_transform_adjoint
from juart.conopt.tfs.fourier import nonuniform_transfer_function
from juart.recon.sense import SENSE
from juart.vis.interactive import InteractiveFigure3D

# Load preprocessed dataset

In [ ]:
store = zarr.storage.LocalStore(
    "/home/jovyan/datasets/num_phantom_sph_traj",
)
group = zarr.open_group(store, mode="r")
k = group["k"][:]
C = group["C"][:]
d = group["d"][:]

# Convert data to JUART format

In [ ]:
# Scale trajectory to [-0.5, 0.5]
print(f"Min/Max of trajectory: {k.min()} / {k.max()}")
k = k / (2 * k.max())
print(f"Min/Max of trajectory: {k.min()} / {k.max()}")

k = torch.from_numpy(k)
C = torch.from_numpy(C)
d = torch.from_numpy(d)

# Perform 3D CG-SENSE reconstruction

In [ ]:
AHd = nonuniform_fourier_transform_adjoint(k, d, (128, 128, 128))
AHd = torch.sum(torch.conj(C) * AHd, dim=0, keepdim=True)

In [ ]:
H = nonuniform_transfer_function(k, (1, 128, 128, 128), oversampling=(2, 2, 2))

In [ ]:
C.shape, AHd.shape, H.shape

In [ ]:
C.dtype, AHd.dtype, H.dtype

In [ ]:
device='cuda:2'

In [ ]:
cg_solver = SENSE(
    C[..., None].to(device),
    AHd[..., None, None].to(device),
    H[..., None, None].to(device),
    axes=(1, 2, 3),
    maxiter=200,
    verbose=True,
    device=device
)

In [ ]:
shape = (128, 128, 128)

In [ ]:
cg_image = cg_solver.solve().view(torch.complex64).reshape(shape)

In [ ]:
InteractiveFigure3D(
    torch.abs(cg_image).cpu().numpy(),
    vmin=0,
    vmax=20,
    title="Reconstructed images",
    cmap="gray",
).interactive

In [ ]:
cg_image.shape